1 - Dans ce notebook nous simulons un réseaux:


    - plusieurs antennes (un agent par antenne) serons dans le réseau


    -plusieurs utilisateurs (users) serons dans le réseau


2 - Chaque agent au sein de chaque antenne sera entrainé localement

3- Nous étudions l'évolution de l'apprentissage des agents par visualisation des courbes d'apprentissage

4- Nous implémentons ici un algorithme d'Apprentissage fédéré !!

In [42]:
import numpy as np
import os
import sys
sys.path.append('../src/')
sys.path.append(os.path.abspath("src"))
from agent import Actor, Critic, ReplayBuffer, DDPGAgent
from env import WirelessEnv
import torch
import rewards_computation
from rewards_computation import compute_rewards

In [43]:
demand_dim = 2       # la dimention de la demande est 2, [demande de puissance, distance à l'antenne]

response_dim = 1     # la dimension de la réponse est 1, c'est la puissance qu'envoie l'antenne en réponse à la demande

max_response = 10    # on limite la puissance allouable par un antenne à 10 : contrainte pour éviter les phénomènes de divergences (c'est comme si les antennes ne pouvaient pas émettre au delà de 10 watt en un seul signal)

env = WirelessEnv(num_antennas=3, num_users=3) # env est une simulation du réseau qui comprends des antennes et des utilisateurs (ici users)


In [44]:
env_state = env._get_state()   # cette méthode renvoie les positions des antennes et des utilisateurs su réseau (le reseau est une grille 100x100)
print(env_state)

{'users': [{'id': 0, 'pos': array([63.61837314, 71.49157505]), 'demand': 5}, {'id': 1, 'pos': array([23.57360115,  0.2148403 ]), 'demand': 10}, {'id': 2, 'pos': array([ 3.5706605 , 33.33540879]), 'demand': 7}], 'antennas': [{'id': 0, 'pos': array([ 0, 50])}, {'id': 1, 'pos': array([20, 50])}, {'id': 2, 'pos': array([40, 50])}]}


In [52]:
Agent_0 = DDPGAgent(demand_dim, response_dim, max_response,buffer_size=1000)
Agent_1 = DDPGAgent(demand_dim, response_dim, max_response,buffer_size=1000)
Agent_2 = DDPGAgent(demand_dim, response_dim, max_response,buffer_size=1000)
# 3 Agents pour les trois antennes

In [46]:
distances_dict = env._user_antennas_distance() # doctionnaire qui contient pour chaque utilisateur la distance qui le separe à chaque antenne.
print(distances_dict)

{0: {0: 67.15046685870627, 1: 48.62561334953669, 2: 31.932982132821127}, 1: {0: 55.0842699676003, 1: 49.913252266101246, 2: 52.42507706904636}, 2: {0: 17.04283475551365, 1: 23.401534064038607, 2: 40.06002217419474}}


In [51]:
matching = {}
for i in range(env.num_users):
    min_antenna = min(distances_dict[i], key=distances_dict[i].get)
    matching[i] = min_antenna
print(matching)


{0: 2, 1: 1, 2: 0}


In [35]:
demandes_per_user_dict = {i: 0 for i in range (env.num_users)}
for i in range(env.num_users):
    demandes_per_user_dict[i] = env.state["users"][i]["demand"]
print(demandes_per_user_dict)
    

{0: 5, 1: 10, 2: 7}


In [ ]:
distances_utiles_per_user = {u["id"]: 0 for u in env.users}
for i in range(env.num_users):
    distances_utiles_per_user[i] = min(distances_dict[i].values() )
print(distances_utiles_per_user) # contient les distances entre chaque user et l'antenne la plus proche soit celle qui lui est assignéé


{0: 47.15089137790735, 1: 49.74692485410736, 2: 10.190328210752103}


In [41]:
demandes_reelles_dict = {i: [demandes_per_user_dict[i], distances_utiles_per_user[i]] for i in range(env.num_users)}
print(demandes_reelles_dict)


{0: [5, 47.15089137790735], 1: [10, 49.74692485410736], 2: [7, 10.190328210752103]}


In [55]:
print(matching)

{0: 2, 1: 1, 2: 0}


In [ ]:
response_dict = {i: 0 for i in range(env.num_antennas)}
for k in range(1000):
    response_2 = Agent_2.select_response(demandes_reelles_dict[0]) 
    response_dict[2]= response_2
    reward_dict = compute_rewards(response_dict,env)
    

    response_1 = Agent_1.select_response(demandes_reelles_dict[1])
    

    response_0 = Agent_0.select_response(demandes_reelles_dict[2])

    

[array([0.], dtype=float32), array([0.49041995], dtype=float32), array([1.0166851], dtype=float32)]
[array([0.], dtype=float32), array([0.9500953], dtype=float32), array([0.], dtype=float32)]
[array([3.0614097], dtype=float32), array([0.18328826], dtype=float32), array([0.26344168], dtype=float32)]
[array([0.], dtype=float32), array([0.], dtype=float32), array([0.51969117], dtype=float32)]
[array([0.], dtype=float32), array([0.70475], dtype=float32), array([0.], dtype=float32)]
[array([0.], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32)]
[array([1.2719918], dtype=float32), array([0.], dtype=float32), array([1.8940774], dtype=float32)]
[array([1.472909], dtype=float32), array([1.2598089], dtype=float32), array([1.3620869], dtype=float32)]
[array([0.00118785], dtype=float32), array([0.], dtype=float32), array([0.], dtype=float32)]
[array([0.39792374], dtype=float32), array([0.09314228], dtype=float32), array([0.88925445], dtype=float32)]
[array([0.], dtype=float32